In [1]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, modified_precision
from nltk.translate.chrf_score import sentence_chrf, corpus_chrf
from nltk.metrics import scores
import scipy.io.wavfile
from IPython.display import Audio
from IPython.display import display
from nltk.stem import *
# from nltk.stem.snowball import SnowballStemmer
from stemming.porter2 import stem
import stemming
from nltk.metrics.scores import recall

from nltk.corpus import stopwords

%matplotlib inline

In [2]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)

In [3]:
smooth_fun = nltk.translate.bleu_score.SmoothingFunction()

In [4]:
from bow_run import *

In [5]:
def get_unpadded_batch(m_dict, x_key, y_key, utt_list, vocab_dict, bow_dict, max_enc, max_dec, input_path='', set_zero_num=500):
    batch_data = {'X':[], 't':[], 'y':[]}
    # -------------------------------------------------------------------------
    # loop through each utterance in utt list
    # -------------------------------------------------------------------------
    for i, u in enumerate(utt_list):
        # ---------------------------------------------------------------------
        #  add X data
        # ---------------------------------------------------------------------
        if x_key == 'sp':
            # -----------------------------------------------------------------
            # for speech data
            # -----------------------------------------------------------------
            # get path to speech file
            utt_sp_path = os.path.join(input_path, "{0:s}.npy".format(u))
            if not os.path.exists(utt_sp_path):
                # for training data, there are sub-folders
                utt_sp_path = os.path.join(input_path,
                                           u.split('_',1)[0],
                                           "{0:s}.npy".format(u))
            if os.path.exists(utt_sp_path):
                x_data = Variable(xp.load(utt_sp_path)[:max_enc])
            else:
                # -------------------------------------------------------------
                # exception if file not found
                # -------------------------------------------------------------
                raise FileNotFoundError("ERROR!! file not found: {0:s}".format(utt_sp_path))
                # -------------------------------------------------------------
        else:
            # -----------------------------------------------------------------
            # for text data
            # -----------------------------------------------------------------
            x_ids = [vocab_dict[x_key]['w2i'].get(w, UNK_ID) for w in m_dict[u][x_key]]
            x_ids = xp.asarray(x_ids, dtype=xp.int32)
            batch_data['X'].append(x_ids[:max_enc])
            # -----------------------------------------------------------------
        # ---------------------------------------------------------------------
        #  add labels
        # ---------------------------------------------------------------------
        if type(m_dict[u][y_key]) == list:
            en_ids = list(set([bow_dict['w2i'].get(w, UNK_ID) for w in m_dict[u][y_key]])-set(range(4)))
        else:
            # dev and test data have multiple translations
            # choose the first one for computing perplexity
            en_ids = list(set([bow_dict['w2i'].get(w, UNK_ID) for w in m_dict[u][y_key][0]])-set(range(4)))
        y_ids = en_ids[:max_dec]
        # ---------------------------------------------------------------------
        if len(x_data) > 0 and len(y_ids) > 0:
            batch_data['X'].append(x_data)
            batch_data['t'].append([y_ids])
            y_data = xp.zeros(len(bow_dict['w2i']), dtype=xp.int32)
            #y_data = -1 * xp.ones(len(bow_dict['w2i']), dtype=xp.int32)
            #set_some_to_0 = np.random.choice(np.array(range(len(bow_dict['w2i'])), dtype="i"), 
            #                                 size=set_zero_num, 
            #                                 replace=False)
            #y_data[set_some_to_0] = 0
            y_data[y_ids] = 1
            y_data[list(range(4))] = -1
            batch_data['y'].append(y_data)
            
    # -------------------------------------------------------------------------
    # end for all utterances in batch
    # -------------------------------------------------------------------------
#     if len(batch_data['X']) > 0 and len(batch_data['y']) > 0:
#         batch_data['X'] = F.pad_sequence(batch_data['X'], padding=PAD_ID)
#         batch_data['y'] = F.pad_sequence(batch_data['y'], padding=PAD_ID)
    return batch_data

In [6]:
# def hmm_get_curr_bow_batch(m_dict, x_key, y_key, utt_list, vocab_dict, bow_dict, max_enc, max_dec, input_path='', set_zero_num=500):
#     batch_data = {'X':[], 't':[], 'y':[]}
#     # -------------------------------------------------------------------------
#     # loop through each utterance in utt list
#     # -------------------------------------------------------------------------
#     for i, u in enumerate(utt_list):
#         # ---------------------------------------------------------------------
#         #  add X data
#         # ---------------------------------------------------------------------
#         if x_key == 'sp':
#             # -----------------------------------------------------------------
#             # for speech data
#             # -----------------------------------------------------------------
#             # get path to speech file
#             utt_sp_path = os.path.join(input_path, "{0:s}.npy".format(u))
#             if not os.path.exists(utt_sp_path):
#                 # for training data, there are sub-folders
#                 utt_sp_path = os.path.join(input_path,
#                                            u.split('_',1)[0],
#                                            "{0:s}.npy".format(u))
#             if os.path.exists(utt_sp_path):
#                 x_data = xp.load(utt_sp_path)[:max_enc]
#             else:
#                 # -------------------------------------------------------------
#                 # exception if file not found
#                 # -------------------------------------------------------------
#                 raise FileNotFoundError("ERROR!! file not found: {0:s}".format(utt_sp_path))
#                 # -------------------------------------------------------------
#         else:
#             # -----------------------------------------------------------------
#             # for text data
#             # -----------------------------------------------------------------
#             x_ids = [vocab_dict[x_key]['w2i'].get(w, UNK_ID) for w in m_dict[u][x_key]]
#             x_ids = xp.asarray(x_ids, dtype=xp.int32)
#             batch_data['X'].append(x_ids[:max_enc])
#             # -----------------------------------------------------------------
#         # ---------------------------------------------------------------------
#         #  add labels
#         # ---------------------------------------------------------------------
#         if type(m_dict[u][y_key]) == list:
#             en_ids = list(set([bow_dict['w2i'].get(w, UNK_ID) for w in m_dict[u][y_key]])-set(range(4)))
#         else:
#             # dev and test data have multiple translations
#             # choose the first one for computing perplexity
#             en_ids = list(set([bow_dict['w2i'].get(w, UNK_ID) for w in m_dict[u][y_key][0]])-set(range(4)))
#         y_ids = en_ids[:max_dec]
#         # ---------------------------------------------------------------------
#         if len(x_data) > 0 and len(y_ids) > 0:
#             batch_data['X'].append(x_data)
#             batch_data['t'].append([y_ids])
#             y_data = xp.zeros(len(bow_dict['w2i']), dtype=xp.int32)
#             #y_data = -1 * xp.ones(len(bow_dict['w2i']), dtype=xp.int32)
#             #set_some_to_0 = np.random.choice(np.array(range(len(bow_dict['w2i'])), dtype="i"), 
#             #                                 size=set_zero_num, 
#             #                                 replace=False)
#             #y_data[set_some_to_0] = 0
#             y_data[y_ids] = 1
#             y_data[list(range(4))] = -1
#             batch_data['y'].append(y_data)
            
#     # -------------------------------------------------------------------------
#     # end for all utterances in batch
#     # -------------------------------------------------------------------------
#     if len(batch_data['X']) > 0 and len(batch_data['y']) > 0:
#         batch_data['X'] = F.pad_sequence(batch_data['X'], padding=PAD_ID)
#         batch_data['y'] = F.pad_sequence(batch_data['y'], padding=PAD_ID)
#     return batch_data

### Fisher dev

In [7]:
cfg_path = "./sp2bagwords/sp_1.0_h-256_e-128_rnn-2_hwy-2_cnn-32-2-5/"

In [8]:
!cd .

In [9]:
last_epoch, model, optimizer, m_cfg, t_cfg = check_model(cfg_path);

cnn_out_dim = rnn_in_units =  640


/afs/inf.ed.ac.uk/group/project/lowres/work/miniconda3/envs/chainer3/lib/python3.6/site-packages/chainer/utils/experimental.py:104: FutureWarning: chainer.links.normalization.layer_normalization.py is experimental. The interface can change in the future.
  FutureWarning)


using ADAM optimizer
--------------------------------------------------------------------------------
model found = 
./sp2bagwords/sp_1.0_h-256_e-128_rnn-2_hwy-2_cnn-32-2-5/seq2seq_20.model
finished loading ..
optimizer found = ./sp2bagwords/sp_1.0_h-256_e-128_rnn-2_hwy-2_cnn-32-2-5/train.opt
finished loading optimizer ...


In [10]:
# bow_dict_path = os.path.join(m_cfg['data_path'],
#                                      'train_top_K_enw.dict')
# if os.path.exists(bow_dict_path):
#     bow_dict = pickle.load(open(bow_dict_path, "rb"))

In [11]:
%%capture

train_key = m_cfg['train_set']
dev_key = m_cfg['dev_set']
batch_size=t_cfg['batch_size']
enc_key=m_cfg['enc_key']
dec_key=m_cfg['dec_key']
input_path = os.path.join(m_cfg['data_path'], m_cfg['dev_set'])
# -------------------------------------------------------------------------
# get data dictionaries
# -------------------------------------------------------------------------
map_dict, vocab_dict, bucket_dict, bow_dict = get_data_dicts(m_cfg)
batch_size = {'max': 96, 'med': 128, 'min': 256, 'scale': 1}

In [12]:
sum(bow_dict['freq'].values()), sum(vocab_dict['en_w']['freq'].values()), sum(bow_dict['freq'].values()) / sum(vocab_dict['en_w']['freq'].values())

(27715, 1497356, 0.018509292379367364)

In [13]:
list(bow_dict['i2w'].keys())[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [14]:
random.seed("meh")
# random.seed("haha")

In [15]:
# Eval parameters
ref_index = -1
min_len, max_len= 0, m_cfg['max_en_pred']
# min_len, max_len = 0, 10
displayN = 50
m_dict=map_dict[dev_key]
# wavs_path = os.path.join(m_cfg['data_path'], "wavs")
wavs_path = os.path.join("../chainer2/speech2text/both_fbank_out/", "wavs")
v_dict = vocab_dict['en_w']
key = m_cfg['dev_set']

### BOW trials

In [16]:
bucket_dict['fisher_dev']['buckets'][0][:1]

['20051009_182032_217_fsp-B-1']

In [17]:
bucket_dict['fisher_dev']['num_b']

10

In [18]:
curr_bucket = 3
num_utts = 30
utt_list = bucket_dict['fisher_train']['buckets'][curr_bucket][:num_utts]
curr_set='fisher_train'

In [19]:
if "train" in curr_set:
    local_input_path = os.path.join(m_cfg['data_path'], m_cfg['train_set'])
    play_audio = False
else:
    local_input_path = os.path.join(m_cfg['data_path'], m_cfg['dev_set'])
    play_audio = True

width_b = bucket_dict[dev_key]["width_b"]

batch_data = get_bow_batch(map_dict[curr_set], 
                                enc_key,
                                dec_key,
                                utt_list,
                                vocab_dict,
                                bow_dict,
                                (curr_bucket+1) * width_b,
                                200,
                                input_path=local_input_path)

X, y, t = batch_data['X'], batch_data['y'], batch_data['t']

batch_size = X.shape[0]

In [20]:
t

[[[17]],
 [[17]],
 [[]],
 [[]],
 [[93]],
 [[25]],
 [[93]],
 [[]],
 [[]],
 [[11]],
 [[]],
 [[]],
 [[]],
 [[]],
 [[51]],
 [[]],
 [[]],
 [[]],
 [[]],
 [[]],
 [[]],
 [[]],
 [[]],
 [[]],
 [[]],
 [[58]],
 [[]],
 [[12]],
 [[40, 4]],
 [[]]]

In [21]:
xp.min(batch_data['X'].data)

array(-4.809871196746826, dtype=float32)

In [22]:
if "train" in curr_set:
    local_input_path = os.path.join(m_cfg['data_path'], m_cfg['train_set'])
    play_audio = False
else:
    local_input_path = os.path.join(m_cfg['data_path'], m_cfg['dev_set'])
    play_audio = True

width_b = bucket_dict[dev_key]["width_b"]

batch_data = get_bow_batch(map_dict[curr_set], 
                                enc_key,
                                dec_key,
                                utt_list,
                                vocab_dict,
                                bow_dict,
                                (curr_bucket+1) * width_b,
                                200,
                                input_path=local_input_path)

X, y, t, l = batch_data['X'], batch_data['y'], batch_data['t'], batch_data['l']

# batch_size = X.shape[0]

In [23]:
print(X.shape, y.shape)

(30, 799, 80) (30, 104)


In [ ]:
# optimizer.hyperparam.alpha = 0.001
optimizer.hyperparam.lr = 1.0

In [ ]:
predicted_out[0,:15]

In [24]:
PRED_THRESH = 0.2

In [25]:
for i in range(5):
    p_words, loss, p_probs = model.forward_bow(X=batch_data['X'],
                                                y=batch_data['y'],
                                                add_noise=t_cfg['speech_noise'],
                                                l=batch_data['l'])
    model.cleargrads()
    loss.backward()
    optimizer.update()
    loss_val = float(loss.data)
    print(loss_val)

0.06361255049705505
0.06104724854230881
0.056551627814769745
0.05156370624899864
0.04425334557890892


In [ ]:
for i in range(1):
    pred_words = []
    batch_data = get_bow_batch(map_dict[curr_set], 
                                enc_key,
                                dec_key,
                                utt_list,
                                vocab_dict,
                                bow_dict,
                                (curr_bucket+1) * width_b,
                                200,
                                input_path=local_input_path)

    X, y, t, l  = batch_data['X'], batch_data['y'], batch_data['t'], batch_data['l']

    batch_size = X.shape[0]
    # encode input
    model.forward_enc(X)
    
    if m_cfg['highway_layers'] > 0:
        highway_h = model.forward_highway(model.h_final_rnn)

    predicted_out = model.out(highway_h)
    
    simple_loss = F.sigmoid_cross_entropy(predicted_out, y, normalize=True, reduce="mean")
    loss = F.sigmoid_cross_entropy(predicted_out, y, normalize=True, reduce="no")
    
    loss_weights = xp.ones(shape=y.data.shape, dtype="f")
    loss_weights[y.data < 0] = 0
    loss_weights[y.data == 0] = 1
    loss_weights[y.data > 0] = 10
    #loss_avg = F.average(F.sigmoid_cross_entropy(predicted_out, y, normalize=True, reduce='no'), weights=loss_weights)
    loss_avg = F.mean(loss_weights * loss)
    print(i, "---".join(["{0:.3f}".format(float(val)) for val in (loss_avg.data, xp.mean(loss.data), simple_loss.data)]))
    model.cleargrads()
    loss_avg.backward()
    optimizer.update()
#     for row in predicted_out.data[:5]:
#         print(xp.where(row > 0.6)[0][:20])

In [ ]:
probs = []
for i, row in enumerate(predicted_out.data):
    print(row[10:12])
    pred_inds = xp.where(row >= PRED_THRESH)[0]
    if len(pred_inds) > 20:
        pred_inds = xp.argsort(row)[-20:][::-1]
    #pred_words.append([bow_dict['i2w'][i] for i in pred_inds.tolist()])
    curr_row = [i for i in pred_inds.tolist() if i > 3]
#     if i < 5:
#         print(curr_row)
    pred_words.append(curr_row)
    probs.append(row)
# print(pred_words)

In [ ]:
probs = F.pad_sequence(probs).data

In [ ]:
probs[:,10:12]

In [ ]:
N = 5
for row, pred, ttt in zip(y.data[:N], predicted_out.data[:N], batch_data['t']):
    #print(xp.where(row == 1))
    print(ttt[0])
    print(xp.where(pred >= PRED_THRESH)[0].tolist())
    print(set(xp.where(pred >= PRED_THRESH)[0].tolist()) & set(ttt[0]))
    print("-"*50)

In [ ]:
basic_precision_recall(batch_data['t'], pred_words)

In [ ]:
F.binary_accuracy(predicted_out.data, y)

In [ ]:
batch_data['y'][:,:20]

In [ ]:
loss_weight = xp.ones(shape=y.data.shape, dtype="f")
loss_weight[y.data < 0] = 0
loss_weight[y.data == 0] = 10
loss_weight[y.data > 0] = 1

In [ ]:
y.data[0][:20], loss_weight[0,:20]

In [ ]:
loss_full = F.sigmoid_cross_entropy(predicted_out, y, normalize=True, reduce='no')

In [ ]:
loss_full.shape, loss_weight.shape, loss_full[0,:12]

In [ ]:
F.average(loss_full, weights=loss_weight), F.mean(loss_full), F.mean((loss_full*loss_weight))

In [ ]:
F.average(F.sigmoid_cross_entropy(predicted_out, y, normalize=True, reduce="no").data), loss

In [ ]:
F.sigmoid_cross_entropy(predicted_out, y, normalize=True, reduce="mean").data

In [ ]:
out_h = model.out_1(model.h_final_rnn)
predicted_out = model.out_2(F.sigmoid(out_h))
loss = F.sigmoid_cross_entropy(predicted_out, y, normalize=True)

In [ ]:
print(loss.data)

In [ ]:
model.cleargrads()
loss.backward()
optimizer.update()

In [ ]:
predicted_out.data[0,9:20]

In [ ]:
print(xp.where(predicted_out.data > 0))

In [ ]:
X.shape, y.shape

In [ ]:
y[0,:50]

In [ ]:
xp.where(y[0] == 1)

In [ ]:
set(t[0])

In [ ]:
[vocab_dict['en_w']['i2w'][i] for i in t[0]]

In [ ]:
model.h_final_rnn.shape

In [ ]:
model.h_final_rnn[-1,:10], model.h_final_rnn[-1,-10:]

In [ ]:
model.L1_enc.h[-1,:10], model.L1_rev_enc.h[-1][-10:]

In [ ]:
h_fwd = model.L1_enc.h.data
h_rev = model.L1_rev_enc.h.data

In [ ]:
h_fwd[-1,:10], h_rev[-1,-10:]

In [ ]:
F.concat((h_fwd, h_rev), axis=1)[-1][:10], F.concat((h_fwd, h_rev), axis=1)[-1][-10:]

In [ ]:
pata = model[model.rnn_enc[-1]].h.data
if model.m_cfg['bi_rnn']:
    h_rev = model[model.rnn_rev_enc[-1]].h.data
    pata = F.concat((pata, h_rev), axis=1)

In [ ]:
pata[-1][-10:]

In [ ]:
model.out_1

In [ ]:
haha = F.sigmoid(model.out_1(model.h_final_rnn))

In [ ]:
from scipy.special import expit

In [ ]:
y.shape

In [ ]:
dummy_y = xp.ones(y.shape, dtype='i')

In [ ]:
hu = model.out_2(haha)

In [ ]:
F.sigmoid_cross_entropy(hu, dummy_y)

In [ ]:
loss = F.sigmoid_cross_entropy(hu, y)
loss

In [ ]:
model.cleargrads()
loss.backward()
optimizer.update()

In [ ]:
ha = F.sigmoid(model.out_2(haha)).data

In [ ]:
[expit(i) for i in hu[0][:10].tolist()]

In [ ]:
ha[0].data[:10]

In [ ]:
xp.where(ha[0].data >= 0)

In [ ]:
loss = F.sigmoid_cross_entropy(model.out_2(haha), y, normalize=True)

In [ ]:
y[0][:13]

In [ ]:
loss

In [ ]:
ha.shape

In [ ]:
ha[0,:10]

In [ ]:
F.sigmoid

In [ ]:
ha.data.shape

In [ ]:
batch_data['y'][0,:5]

In [ ]:
y = np.array([[0.1, 0.7, 0.2], # prediction label is 1
              [8.0, 1.0, 2.0], # prediction label is 0
              [-8.0, 1.0, 2.0], # prediction label is 2
              [-8.0, -1.0, -2.0]]) # prediction label is 1
t = np.array([1, 0, 2, 1], 'i')

In [ ]:
F.accuracy(y, t).data # 100% accuracy because all samples are correct

In [ ]:
t = np.array([1, 0, 0, 0], 'i')
F.accuracy(y, t).data # 100% accuracy because all samples are correct

In [ ]:
ha.shape, batch_data['y'].shape

In [ ]:
F.binary_accuracy(ha.data, batch_data['y'])